In [1]:
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')


rs = 132
dataset = pd.read_csv('./data/dataset.csv')
dataset.set_index('股票简称', inplace=True)
# print(dataset.shape[0])
features = [
    '净资产收益率(%)', '资产报酬率(%)', 'EBITDA率(%)', '营业利润率(%)', '投入资本回报率(%)', 
    '资产负债率(%)', '权益乘数(%)', '速动比率(%)', '现金流动负债比率(%)', '长期资本负债率(%)',
    '营业收入增长率(%)', '资本保值增值率(%)', '总资产增长率(%)', '资本积累率(%)', '营业利润增长率(%)',
    '总资产周转率', '应收账款周转率', '流动资产周转率', '存货周转率', '现金资产比率(%)', 
    '数字技术应用', '商业模式变革', '智能制造', '现代信息系统',
    '客户集中度(%)', '供应商集中度(%)', '成本费用利润率(%)',
    '研发人员占比(%)', '研发营收比(%)', '发明专利申请数',
    '两权分离率(%)', '独董比例(%)', '董事会规模','股权集中度(%)',
    '员工人均营收比(%)', '提供岗位增长率(%)', '员工收入增长率(%)',
]
label_name = '因子得分'
unit_map = dataset[['股票代码', '行业代码', '所属省份']].to_dict()
# 获取数据集和标签值
y : pd.Series = dataset[label_name]
X : pd.DataFrame = dataset[features].copy(deep=True).astype("float")
# 数据预处理：1.极差标准化；2.数据集划分。
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rs)
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)

In [2]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score
plt.rcParams['axes.unicode_minus'] = False
# 一般机器学习模型的训练与优化

# 1.最小二乘线性回归
model = LinearRegression(n_jobs=-1)
model.fit(X_train_s, y_train)
y_pred = model.predict(X_test_s)
print(f"OLS Regression | r2:{r2_score(y_test, y_pred)} | mean_squared_error:{mean_squared_error(y_test, y_pred):.2} | mean_absolute_error:{mean_absolute_error(y_test, y_pred):.2}。\n 因子分析的结果实际是对原结果进行线性转化的过程，因此最小二乘法能够较为精确的计算出实际权重。")

OLS Regression | r2:1.0 | mean_squared_error:4.5e-27 | mean_absolute_error:5.4e-14。
 因子分析的结果实际是对原结果进行线性转化的过程，因此最小二乘法能够较为精确的计算出实际权重。


In [ ]:
from sklearn.tree import DecisionTreeRegressor

# 基于树模型的参数优化基本流程为：首先计算单个参数的大致范围，随后进行网格搜索。
# 决策树算法优化思路:

# 1.优化ccp_alpha(网格搜索)；

# model = DecisionTreeRegressor(random_state=rs)
# path = model.cost_complexity_pruning_path(X_train_s, y_train)
# param_grid = {'ccp_alpha': path.ccp_alphas}
# kfold = KFold(n_splits=10, shuffle=True, random_state=1)
# model = GridSearchCV(DecisionTreeRegressor(random_state=rs), param_grid, cv=kfold)
# model.fit(X_train_s, y_train)
# print(model.best_params_)

# 2.单参数优化；
model = DecisionTreeRegressor(max_features="sqrt", random_state=1)
path = model.cost_complexity_pruning_path(X_train_s, y_train)
param_grid = {
    'ccp_alpha': path.ccp_alphas,  # 剪枝参数
    'max_depth': np.arange(5, 15, 1),  # 决策树最大深度，用来防止过拟合; 
    'min_samples_split': np.arange(1, 3, 1),  # 分裂节点所需的最小样本数，也就是如果样本数小于这个值就不划分了。
    'min_samples_leaf': np.arange(1, 4, 1)  # 叶节点所需的最小样本数，如果样本数小于这个，就不划分了。用来防止过拟合
}
# 2.网格法参数微调
kfold = KFold(n_splits=10, shuffle=True, random_state=rs)
model = GridSearchCV(DecisionTreeRegressor(max_features="sqrt", random_state=rs), param_grid, cv=kfold, n_jobs=-1)
model.fit(X_train_s, y_train)

# 决策树最优超参数：{'ccp_alpha': 0.7814179316082759, 'max_depth': 11, 'min_samples_leaf': 1, 'min_samples_split': 2} ；综合得分0.7849455038096266
print(f"决策树最优超参数：{model.best_params_}\n 综合得分{model.best_estimator_.score(X_test_s, y_test)}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# 随机森林参数优化：https://www.jianshu.com/p/f5b45a60289f
param_grid = {
    'max_depth': np.arange(2, 30, 1),  # 树的最大深度
    'n_estimators': np.arange(2, 30, 1),  # 森林中树的数量
    'min_samples_split': [2, 3, 4, 5],  # 分裂节点所需的最小样本数
    'min_samples_leaf': [1, 2, 3],  # 叶节点所需的最小样本数
    'max_features': np.arange(2, 30, 1)  # 每次分裂时考虑的特征数量
}

model = RandomForestRegressor(verbose=0, random_state=rs, min_samples_split=2, min_samples_leaf=2)
kfold = KFold(n_splits=10, shuffle=True, random_state=rs)
model = GridSearchCV(model, param_grid, cv=kfold, scoring='r2', verbose=0, n_jobs=-1)
model.fit(X_train_s, y_train)
# RandomForest 最优超参数：{'max_depth': 13, 'max_features': 11, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 7}
 # 综合得分0.8483375813154402

print(f"RandomForest 最优超参数：{model.best_params_}\n 综合得分{model.best_estimator_.score(X_test_s, y_test)}")

In [ ]:
from xgboost import XGBRegressor

# 3.xgboost参数优化：https://www.cnblogs.com/showmeai/p/16037327.html；
params = {
   "objective": 'reg:squarederror',
   "n_estimators": 10,
   "max_depth": 9,
   "learning_rate": 0.24,
   "subsample": 0.2,
   "min_child_weight": 1,
   "gamma": 0.1,
   "colsample_bytree": 0.8,
   "reg_alpha": 0,
   "reg_lambda": 0,
   # "scale_pos_weight": 1,
}
param_grid = {
    'n_estimators': np.arange(18, 24, 1),
    'max_depth': np.arange(8, 12, 1),
    'learning_rate': np.arange(0.15, 0.25, 0.01), # [0.05, 0.1, 0.2]
    "min_child_weight": [4, 5, 6, 7],
    # "gamma": [0, 0.1],
    'subsample': np.arange(0.4, 0.7, 0.1),
    'colsample_bytree': np.arange(0.95, 1.2, 0.01),
}

model = XGBRegressor(random_state=rs)
kfold = KFold(n_splits=10, shuffle=True, random_state=rs)
model = GridSearchCV(model, param_grid, cv=kfold, scoring='r2', verbose=0, n_jobs=-1)
model.fit(X_train_s, y_train)  
print(f"XGBRegressor 最优超参数：{model.best_params_}\n 综合得分{model.best_estimator_.score(X_test_s, y_test)}")

# XGBRegressor 最优超参数：{'colsample_bytree': 0.9800000000000001, 'gamma': 0.1, 'learning_rate': 0.2, 'max_depth': 11, 'min_child_weight': 5, 'n_estimators': 23, 'subsample': 0.5}   综合得分0.8801265955806522

In [140]:
# from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# 4.catboost参数优化
param_grid = {
    'iterations': np.arange(100, 401, 30),
    'depth': [1, 2, 8, 12], 
    'learning_rate': [0.01, 0.1, 0.2],
    'l2_leaf_reg': [0, 1],
    'bagging_temperature': [0.0, 0.1, 0.3],
    'border_count': np.arange(32, 128, 10)
}
model = CatBoostRegressor(random_state=rs, verbose=0, train_dir=None, **params)
kfold = KFold(n_splits=10, shuffle=True, random_state=rs)
model = GridSearchCV(model, param_grid, cv=kfold, scoring='r2', verbose=0, n_jobs=-1)
model.fit(X_train_s, y_train) 
print(f"XGBRegressor 最优超参数：{model.best_params_}\n 综合得分{model.best_estimator_.score(X_test_s, y_test)}")

KeyboardInterrupt: 

In [ ]:
params = {
    "iterations": 100,  # 越大越好300
    "learning_rate": 0.2,  # 可能处于 0.01~0.5或0.6~0.9之间，前者可能性更大
    "depth": 5,
    "l2_leaf_reg": 1,
    'bagging_temperature': 0,
    # "border_count": 64  # 越大越好，
}
param_range = np.arange(1, 200, 10)
scores = {}
for i in param_range:
    model = CatBoostRegressor(random_state=rs, verbose=0, train_dir=None, border_count=i, **params)
    model.fit(X_train_s, y_train)
    scores[i] = model.score(X_test_s, y_test)

temp_data = pd.Series(scores)
# _ind = min(scores, key=scores.get)
plt.figure(figsize=[20, 5])
temp_data.plot()
plt.show()

In [ ]:
from utils.mlkits import r2_score
from tensorflow import keras

# 神经网络模型训练与优化

# Sequential 模型适用于普通层堆栈 其中，每层只有一个 input Tensor 和一个 Output Tensor。
model = keras.models.Sequential()
# model.add(keras.Input(shape=(X_train_s.shape[1], )))
model.add(keras.layers.Dense(units=15, activation='relu', name="layer1", input_shape=(X_train_s.shape[1], ),
                               kernel_regularizer=keras.regularizers.l2(0.02)))
model.add(keras.layers.Dropout(0.08))

# 增加输出层
model.add(keras.layers.Dense(units=1, name="output"))
optimizer = keras.optimizers.Adam(learning_rate=9e-3)
model.compile(optimizer=optimizer, loss='mse', metrics=['mse', r2_score, 'mae'])
# 查看模型结构
# utils.plot_model(model, "./assert/feature_importance/bp_model_structure.png", show_shapes=True)
# model.summary()
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
hist = model.fit(X_train_s, y_train, validation_split=0.3, epochs=300, batch_size=30, shuffle=False, verbose=0,
                         callbacks=[early_stopping])
# validation_split是训练集验证集拆分，epochs代表训练300轮，batch_size代表在批量梯度下降时每次选择16个样本，shuffle代表在训练过程中不会将数据反复打乱
# verbose：日志显示，0为不在标准输出流输出日志信息，1为输出进度条记录，2为每个epoch输出一行记录。
s = model.evaluate(X_test_s, y_test, verbose=0)
print(f'测试集mse得分:{s[1]:.3%}，R^2得分{s[2]:.3%}')


model.save("./assert/temp/bp_keras_model.keras")
pd.DataFrame(hist.history).to_csv('./assert/bp_model_loss.csv')